In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
#%matplotlib inline
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize
import os 
directory = r'E:\video classification'
os.chdir(directory) 
print(os.listdir(directory))
count = 0
videoFile = "Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")



count = 0
videoFile = "Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")
data = pd.read_csv('mapping.csv')
test = pd.read_csv('testing.csv')
print ("1")
X = []
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)
print ("2")
from keras.utils import np_utils
train_y = np_utils.to_categorical(data.Class)
test_y = np_utils.to_categorical(test.Class)

image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)
print ("3")
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')
test_image = preprocess_input(test_image, mode='tf')

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
test_image = base_model.predict(test_image)

X_train = X_train.reshape(208, 7*7*512)
X_valid = X_valid.reshape(90, 7*7*512)
test_image = test_image.reshape(186, 7*7*512)
print ("4")
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
test_image = test_image/test_image.max()

model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(3, activation='softmax'))            # output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print ("5")
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Class), data.Class)  # computing weights of different classes

from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss

model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)
print ("6")
model.load_weights("weights.best.hdf5")

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

scores = model.evaluate(test_image, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print ("7")

Using TensorFlow backend.


['frame0.jpg', 'frame1.jpg', 'frame10.jpg', 'frame100.jpg', 'frame101.jpg', 'frame102.jpg', 'frame103.jpg', 'frame104.jpg', 'frame105.jpg', 'frame106.jpg', 'frame107.jpg', 'frame108.jpg', 'frame109.jpg', 'frame11.jpg', 'frame110.jpg', 'frame111.jpg', 'frame112.jpg', 'frame113.jpg', 'frame114.jpg', 'frame115.jpg', 'frame116.jpg', 'frame117.jpg', 'frame118.jpg', 'frame119.jpg', 'frame12.jpg', 'frame120.jpg', 'frame121.jpg', 'frame122.jpg', 'frame123.jpg', 'frame124.jpg', 'frame125.jpg', 'frame126.jpg', 'frame127.jpg', 'frame128.jpg', 'frame129.jpg', 'frame13.jpg', 'frame130.jpg', 'frame131.jpg', 'frame132.jpg', 'frame133.jpg', 'frame134.jpg', 'frame135.jpg', 'frame136.jpg', 'frame137.jpg', 'frame138.jpg', 'frame139.jpg', 'frame14.jpg', 'frame140.jpg', 'frame141.jpg', 'frame142.jpg', 'frame143.jpg', 'frame144.jpg', 'frame145.jpg', 'frame146.jpg', 'frame147.jpg', 'frame148.jpg', 'frame149.jpg', 'frame15.jpg', 'frame150.jpg', 'frame151.jpg', 'frame152.jpg', 'frame153.jpg', 'frame154.jpg', '

Epoch 4/100
208/208 [==============================] - 5s 24ms/step - loss: 1.1049 - accuracy: 0.4135 - val_loss: 0.9623 - val_accuracy: 0.5667

Epoch 00004: val_loss improved from 1.02293 to 0.96235, saving model to weights.best.hdf5
Epoch 5/100
208/208 [==============================] - 5s 25ms/step - loss: 0.9438 - accuracy: 0.5817 - val_loss: 0.8978 - val_accuracy: 0.5444

Epoch 00005: val_loss improved from 0.96235 to 0.89781, saving model to weights.best.hdf5
Epoch 6/100
208/208 [==============================] - 6s 30ms/step - loss: 0.8919 - accuracy: 0.6202 - val_loss: 0.7917 - val_accuracy: 0.6667

Epoch 00006: val_loss improved from 0.89781 to 0.79168, saving model to weights.best.hdf5
Epoch 7/100
208/208 [==============================] - 6s 27ms/step - loss: 0.6886 - accuracy: 0.7404 - val_loss: 0.7022 - val_accuracy: 0.7000

Epoch 00007: val_loss improved from 0.79168 to 0.70221, saving model to weights.best.hdf5
Epoch 8/100
208/208 [==============================] - 6s 27

208/208 [==============================] - 6s 30ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.4741 - val_accuracy: 0.8556

Epoch 00043: val_loss did not improve from 0.32242
Epoch 44/100
208/208 [==============================] - 6s 27ms/step - loss: 0.0214 - accuracy: 0.9952 - val_loss: 0.3797 - val_accuracy: 0.8778

Epoch 00044: val_loss did not improve from 0.32242
Epoch 45/100
208/208 [==============================] - 5s 26ms/step - loss: 0.0090 - accuracy: 0.9952 - val_loss: 0.3679 - val_accuracy: 0.8889

Epoch 00045: val_loss did not improve from 0.32242
Epoch 46/100
208/208 [==============================] - 6s 27ms/step - loss: 0.0231 - accuracy: 0.9952 - val_loss: 0.4076 - val_accuracy: 0.8667

Epoch 00046: val_loss did not improve from 0.32242
Epoch 47/100
208/208 [==============================] - 6s 28ms/step - loss: 0.0180 - accuracy: 0.9904 - val_loss: 0.4268 - val_accuracy: 0.8667

Epoch 00047: val_loss did not improve from 0.32242
Epoch 48/100
208/208 [======


Epoch 00084: val_loss did not improve from 0.32242
Epoch 85/100
208/208 [==============================] - 7s 35ms/step - loss: 0.0079 - accuracy: 0.9952 - val_loss: 0.4913 - val_accuracy: 0.8667

Epoch 00085: val_loss did not improve from 0.32242
Epoch 86/100
208/208 [==============================] - 5s 26ms/step - loss: 0.0096 - accuracy: 0.9952 - val_loss: 0.4497 - val_accuracy: 0.8667

Epoch 00086: val_loss did not improve from 0.32242
Epoch 87/100
208/208 [==============================] - 5s 26ms/step - loss: 0.0279 - accuracy: 0.9904 - val_loss: 0.4580 - val_accuracy: 0.8667

Epoch 00087: val_loss did not improve from 0.32242
Epoch 88/100
208/208 [==============================] - 5s 26ms/step - loss: 0.0054 - accuracy: 0.9952 - val_loss: 0.4163 - val_accuracy: 0.8889

Epoch 00088: val_loss did not improve from 0.32242
Epoch 89/100
208/208 [==============================] - 5s 26ms/step - loss: 0.0129 - accuracy: 0.9904 - val_loss: 0.4096 - val_accuracy: 0.8889

Epoch 00089: v